# 2.4 Blog
- title: 2.4 Blog
- toc: true
- layout: base
- badges: true
- categories: [. Practice]
- description: This is my 2.4 blog

# 2.4a Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

## Create Database

In [34]:
"""
These imports define the key objects
"""
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
"""
These object and definitions are used throughout the Jupyter Notebook.
"""
# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///countries.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()
# This belongs in place where it runs once per project
db.init_app(app)

""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class Country(db.Model):
    __tablename__ = 'countries'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _country = db.Column(db.String(255), unique=False, nullable=False)
    _cid = db.Column(db.String(255), unique=True, nullable=False)
    _continent = db.Column(db.String(255), unique=False, nullable=False)
    _population = db.Column(db.Integer, unique=False, nullable=False)
    #_population = db.Column(db.Integer, unique=False, nullable=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, country, cid, continent, population):
        self._country = country    # variables with self prefix become part of the object, 
        self._cid = cid
        self._continent = continent
        self._population = population
        #self._population = population

    # a name getter method, extracts name from object
    @property
    def country(self):
        return self._country
    
    # a setter function, allows name to be updated after initial object creation
    @country.setter
    def country(self, country):
        self._country = country
    
    # a getter method, extracts uid from object
    @property
    def cid(self):
        return self._cid
    
    # a setter function, allows uid to be updated after initial object creation
    @cid.setter
    def cid(self, cid):
        self._cid = cid
        
    # check if uid parameter matches user id in object, return boolean
    def is_cid(self, cid):
        return self._cid == cid
    
    @property
    def continent(self):
        return self._continent
    
    # a setter function, allows name to be updated after initial object creation
    @continent.setter
    def continent(self, continent):
        self._continent = continent
    
    @property
    def population(self):
        return self._population
    
    # a setter function, allows name to be updated after initial object creation
    @population.setter
    def population(self, population):
        self._population = population

    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "country": self.country,
            "cid": self.cid,
            "continent": self.continent,
            "population": self.population,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, country="", continent="", population=""):
        """only updates values with length"""
        if len(country) > 0:
            self.country = country
        if len(continent) > 0:
            self.continent(continent)
        if len(population) > 0:
            self.population(population)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    
"""Database Creation and Testing """


# Builds working data for testing
def initCountries():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = Country(country='United States of America', cid='US', continent='North America', population=9)
        u2 = Country(country='England', cid='Eng', continent='Europe', population=331900000)
        u3 = Country(country='Canada', cid='Can', continent='North America', population=38250000)
        u4 = Country(country='Mexico', cid='Mex', continent='North America', population=126700000)
        u5 = Country(country='Egypt', cid='Egy', continent='Africa', population=109300000)
        u6 = Country(country='China', cid='Chi', continent='Asia', population=1412000000)


        countries = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for country in countries:
            try:
                '''add user to table'''
                object = country.create()
                print(f"Created new cid {object.cid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist cid {country.cid}, or error.")
                
initCountries()
    
# SQLAlchemy extracts single user from database matching User ID
def find_by_cid(cid):
    with app.app_context():
        place = Country.query.filter_by(_cid=cid).first()
    return place # returns user object

# Check credentials by finding user and verify password
def check_credentials(cid, continent):
    # query email and return user record
    place = find_by_cid(cid)
    if place == None:
        return False
    if (place.is_continent(continent)):
        return True
    return False

def find_by_country(country):
    with app.app_context():
        place = Country.query.filter_by(_country=country).first()
    return place # returns user object

# Check credentials by finding user and verify password
def check_credentials(country):
    # query email and return user record
    place = find_by_cid(country)
    if place == None:
        return False

Created new cid US
Created new cid Eng
Created new cid Can
Created new cid Mex
Created new cid Egy
Created new cid Chi


## Create

In [11]:
      
#check_credentials("indi", "123qwerty")
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    cid = input("Enter your user cid:")
    place = find_by_cid(cid)
    try:
        print("Found\n", place.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    country = input("Enter the country:")
    continent = input("Enter the continent")
    population = input("Enter the population'")

    # Initialize User object before date
    place = Country(country=country, 
                cid=cid, 
                continent=continent,
                population=population
                )
    
    # create user.dob, fail with today as dob

  
    # write object to database
    with app.app_context():
        try:
            object = place.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error cid {cid}")
        
create()

Found
 {'id': 1, 'country': 'United States of America', 'cid': 'US', 'continent': 'North America', 'population': 9}


## Read Database

In [12]:
# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Country.query.all()
    json_ready = [place.read() for place in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'country': 'United States of America',
  'cid': 'US',
  'continent': 'North America',
  'population': 9},
 {'id': 2,
  'country': 'England',
  'cid': 'Eng',
  'continent': 'Europe',
  'population': 331900000},
 {'id': 3,
  'country': 'Canada',
  'cid': 'Can',
  'continent': 'North America',
  'population': 38250000},
 {'id': 4,
  'country': 'Mexico',
  'cid': 'Mex',
  'continent': 'North America',
  'population': 126700000},
 {'id': 5,
  'country': 'Egypt',
  'cid': 'Egy',
  'continent': 'Africa',
  'population': 109300000},
 {'id': 6,
  'country': 'China',
  'cid': 'Chi',
  'continent': 'Asia',
  'population': 1412000000},
 {'id': 7,
  'country': 'France',
  'cid': 'Fre',
  'continent': 'Europe',
  'population': 8438},
 {'id': 8, 'country': '', 'cid': '', 'continent': '', 'population': ''}]

## Update

In [35]:
def update():
    # optimize user time to see if uid exists
    cid = input("Enter your cid:")
    place = find_by_cid(cid)

    if place is None:
        print(f"User {cid} is not found")
        return

    new_country = input("What is your new country: ")
    new_continent = input("What is your new continent: ")
    new_population = input("What is your new population: ")

    with app.app_context():
        try:
            place.update(new_country, new_continent, new_population)
            print(f"Cid, {cid}, has been updated with the country, {new_country}, with the continent {new_continent}, and the population, {new_population}")
        except:
            print(f"There was a problem in updating the country, {cid}")
        
update()

There was a problem in updating the country, Chi


## Delete

In [25]:
def delete():
    # optimize user time to see if uid exists
    cid = input("Enter your cid:")
    place = find_by_cid(cid)

    if place is None:
        print(f"Country, {cid} is not found :(")

    with app.app_context():
        try:
            place.delete()
            print(f"Country, {cid} has been deleted.")
        except:
            print("Enter a country link that already exists")        
delete()

Country, Can has been deleted.


## Menu

In [33]:
# Menu, to run other cells from one control point
def menu():
    operation = input("Enter: (C)reate (R)ead (U)pdate or (D)elete or (S)chema")
    if operation.lower() == 'c':
        create()
    elif operation.lower() == 'r':
        read()
    elif operation.lower() == 'u':
        update()
    elif operation.lower() == 'd':
        delete()
    elif operation.lower() == 's':
        schema()
    elif len(operation)==0: # Escape Key
        return
    else:
        print("Please enter c, r, u, or d") 
    menu() # recursion, repeat menu
        
try:
    menu() # start menu
except:
    print("Perform Jupyter 'Run All' prior to starting menu")

Created
 {'id': 11, 'country': 'Brazil', 'cid': 'Bra', 'continent': 'South America', 'population': 912847219}
Country, Bra has been deleted.
There was a problem in updating the country, US


## 2.4b Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- In this implementation, do you see procedural abstraction?
- In 2.4a or 2.4b lecture
    - Do you see data abstraction?  Complement this with Debugging example.
        - Yes I do see data abstraction because by usuing this form of databases, we are narrowing down the code we have to use. In fact, for the users technecally we don't have to use any code. We can let the website users add their names and accounts and create all the data for us. Also, we can choose one row or column from the table to only read or alter one piece of the data. 
        - ![Debugging](../images/Debug.png)
            - This piece of code creates a new user

        - ![Debugging](../images/Debug2.png)
            - This puts the different types of data into the table

        - ![Debugging](../images/Debug3.png)
            - This code creates the properties of the table. It has setters and getters and is very important because there would be no table rows or columns without this piece of code.

        - ![Debugging](../images/Debug4.png)
            - This piece of code also helps create properties. It decides if a property is going to be unique or not. So a uid would be unique because it is like a username and every user has a different username. 
            
        - ![Debugging](../images/Debug5.png)
            - This piece of code actually creates the database file
    - Use Imperative or OOP style to Create a new Table or do something that applies to your CPT project.
    

Reference... [sqlite documentation](https://www.sqlitetutorial.net/sqlite-python/creating-tables/)
